In [2]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus
from helpers import *
from collections import OrderedDict

In [3]:
DIR = 'TEST_DATA/'
Q_SEED = 10598 # as used in most issues
aqua_globals.random_seed = Q_SEED
shots = 4096
qps = createModelsFromDir(DIR)
optimizer = COBYLA() 
cplex = CplexOptimizer()
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,
                                   seed_simulator=Q_SEED,
                                   seed_transpiler=Q_SEED,
                                   shots=shots) 

In [4]:
qaoa_mes = QAOA(quantum_instance=quantum_instance, optimizer = optimizer)


qaoa = MinimumEigenOptimizer(qaoa_mes)

In [5]:
results = OrderedDict()
for qp_name, qp in qps.items() :
    print(qp_name)
    print("number of qubits: ",qp.qubo_model.get_num_vars())
    res = qaoa.solve(qp)    
    results[qp_name] = res
    if res.status == OptimizationResultStatus.SUCCESS:
        print("success")
        cres  = cplex.solve(qp)
        if res.fval != cres.fval:
            print("\n optimal value QAOA   "+str(res.fval) + "  ,  cplex:"+ str(cres.fval))

# see Class SubstituteVariables in https://qiskit.org/documentation/_modules/qiskit/optimization/problems/quadratic_program.html#QuadraticProgram
# for logger warning constraint A0_leb0 is infeasible due to substitution
# 

test_3
success
test_4
success
test_5


constraint A0_leb0 is infeasible due to substitution


success
test_6
success

 optimal value QAOA   0.0  ,  cplex:-3.0
test_7
success

 optimal value QAOA   -3.0  ,  cplex:-6.0
test_8


NameError: name 'BasicAerError' is not defined

In [6]:
res_5 = results["test_5"]
print(res_5)

optimal function value: -2.0
optimal value: [1. 0. 0. 0. 0.]
status: SUCCESS


In [7]:
qps["test_5"].to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: test_5
// single vars section
dvar bool x0_0;
dvar bool x0_2;
dvar bool x0_3;
dvar bool x0_4;
dvar bool x0_1;

minimize
 - 2 x0_0 + 2 x0_2 + x0_3 + x0_4;
 
subject to {
 A0_leb0:
  x0_0 + 3 x0_2 - 2 x0_3 - x0_1 <= 4;
 A0_leb1:
  4 x0_0 + 4 x0_2 + 3 x0_3 + 4 x0_4 + 4 x0_1 <= 16;
 A0_leb2:
  2 x0_0 + 5 x0_2 - 4 x0_3 + 2 x0_4 + 4 x0_1 <= 5;

}
